<a href="https://colab.research.google.com/github/luciacasass/UFV-VisualizacionDatos/blob/main/EjerciciosClase/Clase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Visualización de Datos

Lucía Casas Sierra

### Exploración inicial

In [24]:
# Importación de librerías
import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")


In [25]:
# Carga de datos
df = pd.read_csv('datos_ejercicio_ventas.csv')
print(df.head())

countries = df['COUNTRY'].unique()
brands = df['SUBBRAND'].unique()

df['DATE'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01')
date_range_actual = [df['DATE'][df['SCENARIO']=='actual'].dt.date.min().strftime('%Y-%m'),
                     df['DATE'][df['SCENARIO']=='actual'].dt.date.max().strftime('%Y-%m')]
date_range_forecast = [df['DATE'][df['SCENARIO']=='AI_forecast'].dt.date.min().strftime('%Y-%m'),
                       df['DATE'][df['SCENARIO']=='AI_forecast'].dt.date.max().strftime('%Y-%m')]

print('\nLista de países:\n', countries)
print('\nLista de marcas:\n', brands)
print()
print('Espacio temporal para datos reales de ventas: ', date_range_actual)
print('Espacio temporal para predicciones de ventas: ', date_range_forecast)

         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT  
0         2023.0  754356.237194  
1         2023.0  560030.558029  
2         2023.0   88501.980847  
3         2023.0  363224.511516  
4         2023.0  396176.120491  

Lista de países:
 ['Portugal' 'Great Britain' 'Spain' 'Hungary' 'Norway' 'Denmark'
 'Netherlands' 'Italy' 'Czech']

Lista de marcas:
 ['Lipton (L3)' 'Pepsi Max (L3)' '7up (L3)' 'Pepsi Regular (L3)'
 'Mountain Dew (L3)' '7up Free (L3)']

Espacio temporal para datos reales de ventas:  ['2023-01', '2024-08']
Espacio temporal para predicciones de ventas:  ['2023-01', '2

In [26]:
# Resumen Ventas Reales vs Predicciones
df_actual = df[df['SCENARIO'] == 'actual']
df_forecast = df[df['SCENARIO'] == 'AI_forecast']

print('VENTAS REALES:')
print('Número de filas: ', len(df_actual))
print(f'Representa el {(len(df_actual)/len(df)*100):.2f}% del archivo original')

print('\nDATOS DE PRONÓSTICO:')
print('Número de filas: ', len(df_forecast))
print(f'Representa el {(len(df_forecast)/len(df)*100):.2f}% del archivo original')

VENTAS REALES:
Número de filas:  900
Representa el 4.82% del archivo original

DATOS DE PRONÓSTICO:
Número de filas:  17766
Representa el 95.18% del archivo original


#### Horizontes de predicción

Se procede al cálculo del horizonte de predicción para los datos de pronóstico disponibles para cada marca por país.

In [27]:
# Se extrae el mes en el que se registró la predicción
df_forecast['FORECAST_MONTH'] = pd.to_numeric(df_forecast['FORECAST'].str.extract('-(\d+)')[0])
# Los campos que no contienen información numérica se refieren a las predicciones registradas en el primer mes
df_forecast['FORECAST_MONTH'] = df_forecast['FORECAST_MONTH'].fillna(1).astype(int)
df_forecast['FORECAST_YEAR'] = df_forecast['FORECAST_YEAR'].astype(int)

# Se completa
df_forecast['FORECAST_DATE'] = pd.to_datetime(
    df_forecast['FORECAST_YEAR'].astype(str) + '-' +
    df_forecast['FORECAST_MONTH'].astype(str) + '-01'
)
# De acuerdo al conocimiento de la base de datos, se resta un mes
df_forecast['FORECAST_DATE'] = df_forecast['FORECAST_DATE'] - pd.DateOffset(months=1)

# Horizonte (meses) = fecha para predicción - fecha de registro
df_forecast['HORIZON_MONTHS'] = (
    (df_forecast['DATE'].dt.year - df_forecast['FORECAST_DATE'].dt.year) * 12
    + (df_forecast['DATE'].dt.month - df_forecast['FORECAST_DATE'].dt.month)
)

print(df_forecast.head())


         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT       DATE  FORECAST_MONTH FORECAST_DATE  \
0           2023  754356.237194 2023-12-01               1    2022-12-01   
1           2023  560030.558029 2023-12-01               1    2022-12-01   
2           2023   88501.980847 2023-12-01               1    2022-12-01   
3           2023  363224.511516 2024-12-01               1    2022-12-01   
4           2023  396176.120491 2023-09-01               1    2022-12-01   

   HORIZON_MONTHS  
0              12  
1              12  
2              12  
3              24  
4               

In [28]:
# Extracción del horizonte máximo por cada país y marca
subbrand_horizon_months = df_forecast.groupby(['COUNTRY', 'SUBBRAND'])['HORIZON_MONTHS'].max().reset_index()
print('Horizonte de predicción para los pares Country + Subbrand: ', subbrand_horizon_months['HORIZON_MONTHS'].unique())
print(subbrand_horizon_months.head())


Horizonte de predicción para los pares Country + Subbrand:  [29]
  COUNTRY            SUBBRAND  HORIZON_MONTHS
0   Czech            7up (L3)              29
1   Czech         Lipton (L3)              29
2   Czech   Mountain Dew (L3)              29
3   Czech      Pepsi Max (L3)              29
4   Czech  Pepsi Regular (L3)              29


### Distribución de ventas reales
#### Por país

In [29]:
# Agrupar por país, sumando el total de ventas
sales_by_country = df_actual.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
sales_by_country.columns = ['COUNTRY', 'TOTAL_SALES']

# Crear el gráfico de barras apiladas
fig = px.bar(
    sales_by_country,
    x='COUNTRY',
    y='TOTAL_SALES',
    color_discrete_sequence=['#489fb5'],
    title='Total Sales by Country'
)

fig.show()

#### Por país y marca

In [30]:
# Agrupar por país y submarca, sumando el total de ventas
sales_by_country_subbrand = df_actual.groupby(['COUNTRY', 'SUBBRAND'])['AMOUNT'].sum().reset_index()

# Crear el gráfico de barras apiladas
fig = px.bar(
    sales_by_country_subbrand,
    x='COUNTRY',
    y='AMOUNT',  # Mostrar el total de ventas
    color='SUBBRAND',  # Usar el color para distinguir subbrands
    color_discrete_sequence=px.colors.qualitative.Pastel,
    title='Total Sales by Brand in Each Country',
    labels={'AMOUNT': 'Total Sales', 'COUNTRY': 'Country', 'SUBBRAND': 'Brand'}
)

fig.show()


#### Por mes y año

In [31]:
# Agrupar por fecha (mes + año), sumando el total de ventas
sales_by_month_year = df_actual.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Renombrar columnas
sales_by_month_year.columns = ['YEAR', 'MONTH', 'TOTAL_SALES']

# Create a new column for easy display of month and year together (optional, for better labeling)
sales_by_month_year['YEAR_MONTH'] = sales_by_month_year['YEAR'].astype(str) + '-' + sales_by_month_year['MONTH'].astype(str).str.zfill(2)

fig = px.bar(
    sales_by_month_year,
    x='YEAR_MONTH',
    y='TOTAL_SALES',
    color_discrete_sequence=['#489fb5'],
    title='Total Sales by Year and Month',
    barmode='group',
    height=600,
    labels={'AMOUNT': 'Total Sales', 'YEAR_MONTH': 'Year-Month'}
)

fig.show()

#### Por marca

In [32]:
# Agrupar por marca, sumando el total de ventas
sales_by_subbrand = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
sales_by_subbrand.columns = ['SUBBRAND', 'TOTAL_SALES']

# Create a bar graph using Plotly Express
fig = px.bar(
    sales_by_subbrand,
    x='SUBBRAND',
    y='TOTAL_SALES',
    color_discrete_sequence=['#489fb5'],
    title='Total Sales by Brand',
    labels={'AMOUNT': 'Total Sales', 'SUBBRAND': 'Brand'}
)

# Show the plot
fig.show()

### Tendencia y estacionalidad
#### Todas las ventas del país con menos ventas

In [33]:
# Identificación del país con menor número de ventas
min_sales_country = sales_by_country.loc[sales_by_country['TOTAL_SALES'].idxmin(), 'COUNTRY']
print('País con menor número de ventas: ', min_sales_country)

País con menor número de ventas:  Spain


In [34]:
sales_by_country_month = df_actual.groupby(['COUNTRY', pd.Grouper(key='DATE', freq='M')])['AMOUNT'].sum().reset_index()
sales_by_country_month.columns = ['COUNTRY', 'DATE', 'TOTAL_SALES']

# Filtrar por el país con ventas mínimas (asegúrate de que min_sales_country esté definido)
country_data = sales_by_country_month[sales_by_country_month['COUNTRY'] == min_sales_country]
country_data['DATE_NUM'] = country_data['DATE'].map(pd.Timestamp.timestamp)

# Crear la cuadrícula de subplots 2x2
fig = make_subplots(rows=2, cols=2, subplot_titles=[f'Tendency Degree {i+1}' for i in range(4)])

# Graficar para cada grado de polinomio en cada subplot
for i in range(4):
    # Realizar la regresión polinómica para el grado actual
    coef = np.polyfit(country_data['DATE_NUM'], country_data['TOTAL_SALES'], i+1)
    polynomial = np.poly1d(coef)

    # Calcular la tendencia para el grado actual
    country_data[f'TENDENCIA_{i+1}'] = polynomial(country_data['DATE_NUM'])

    # Añadir la serie de ventas reales al subplot actual
    fig.add_trace(
        go.Scatter(
            x=country_data['DATE'],
            y=country_data['TOTAL_SALES'],
            mode='lines+markers',
            name='Sales',
            marker=dict(color='#457b9d'),
            showlegend=i == 0  # Mostrar leyenda solo en el primer gráfico
        ),
        row=(i // 2) + 1,
        col=(i % 2) + 1
    )

    # Añadir la línea de tendencia polinómica al subplot actual
    fig.add_trace(
        go.Scatter(
            x=country_data['DATE'],
            y=country_data[f'TENDENCIA_{i+1}'],
            mode='lines',
            name=f'Tendency Degree {i+1}',
            line=dict(dash='dash', color='#8ecae6'),
            showlegend=i == 0
        ),
        row=(i // 2) + 1,
        col=(i % 2) + 1
    )

# Configuración general del diseño
fig.update_layout(
    title_text=f'Sales Tendency Analysis in {min_sales_country}',
    height=800,
    width=1000,
    template='plotly_white'
)

# Configurar etiquetas de ejes y rotación
fig.update_xaxes(title_text="Month", tickangle=45)
fig.update_yaxes(title_text="Sales")

fig.show()

In [35]:
from scipy.fftpack import fft

# Extract the sales data into a numpy array
sales = np.array(country_data['TOTAL_SALES'])

# Perform FFT
fft_values = fft(sales)

# Calculate frequencies corresponding to the FFT values
n = len(sales)
frequencies = np.fft.fftfreq(n)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=frequencies[:n // 2],
        y=np.abs(fft_values[:n // 2]),
        mode='lines',
        line=dict(color='#457b9d'),
        name='Magnitude')
)

fig.update_layout(
    title='Fourier Transform - Frequency Analysis',
    xaxis=dict(title='Frequency'),
    yaxis=dict(title='Magnitude'),
    template='plotly_white',
    width=800,
    height=500
)

fig.show()

In [36]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Asegúrate de que el índice sea de tipo datetime
country_data['DATE'] = pd.to_datetime(country_data['DATE'])

# Establecer la fecha como índice
country_data.set_index('DATE', inplace=True)

# Realizar la descomposición estacional
result = seasonal_decompose(country_data['TOTAL_SALES'], model='additive', period=10)

# Graficar los componentes
# result.plot()
# plt.tight_layout()
# plt.show()

# Crear la figura con subplots para cada componente
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=['Original Series', 'Trend', 'Seasonal', 'Residual'])

fig.add_trace(
    go.Scatter(x=country_data.index, y=country_data['TOTAL_SALES'], mode='lines', name='Original Series'),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=country_data.index, y=result.trend, mode='lines', name='Trend'),
    row=2, col=1)

fig.add_trace(
    go.Scatter(x=country_data.index, y=result.seasonal, mode='lines', name='Seasonal'),
    row=3, col=1)

fig.add_trace(
    go.Scatter(x=country_data.index, y=result.resid, mode='markers', name='Residual', marker=dict(color='green')),
    row=4, col=1)

# Configuración del layout del gráfico
fig.update_layout(
    title='Seasonal Decomposition of Total Sales',
    height=800,
    template='plotly_white')

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Sales", tickformat=".2f")
fig.show()

#### La marca con más ventas

In [37]:
# Identificación del país con mayor número de ventas
sales_by_brand_month = df_actual.groupby(['SUBBRAND', pd.Grouper(key='DATE', freq='M')])['AMOUNT'].sum().reset_index()
sales_by_brand_month.columns = ['SUBBRAND', 'DATE', 'TOTAL_SALES']
max_sales_brand = sales_by_brand_month.loc[sales_by_brand_month['TOTAL_SALES'].idxmax(), 'SUBBRAND']
print('Marca con mayor número de ventas: ', max_sales_brand)

Marca con mayor número de ventas:  Pepsi Max (L3)


In [38]:
sales_by_brand_month = df_actual.groupby(['SUBBRAND', pd.Grouper(key='DATE', freq='M')])['AMOUNT'].sum().reset_index()
sales_by_brand_month.columns = ['SUBBRAND', 'DATE', 'TOTAL_SALES']

# Filtrar por el país con ventas mínimas (asegúrate de que min_sales_country esté definido)
brand_data = sales_by_brand_month[sales_by_brand_month['SUBBRAND'] == max_sales_brand]
brand_data['DATE_NUM'] = brand_data['DATE'].map(pd.Timestamp.timestamp)

# Crear la cuadrícula de subplots (2x2)
fig = make_subplots(rows=2, cols=2, subplot_titles=[f'Tendency Degree {i+1}' for i in range(4)])

# Graficar para cada grado de polinomio
for i in range(4):
    # Realizar la regresión polinómica para el grado actual
    coef = np.polyfit(brand_data['DATE_NUM'], brand_data['TOTAL_SALES'], i + 1)
    polynomial = np.poly1d(coef)

    # Calcular la tendencia para el grado actual
    brand_data['TENDENCIA'] = polynomial(brand_data['DATE_NUM'])

    # Seleccionar la fila y columna correctas en la cuadrícula
    row = (i // 2) + 1
    col = (i % 2) + 1

    # Agregar las ventas reales al subplot
    fig.add_trace(
        go.Scatter(
            x=brand_data['DATE'],
            y=brand_data['TOTAL_SALES'],
            mode='markers+lines',
            name='Sales',
            marker=dict(color='#457b9d'),
            line=dict(color='#457b9d'),
            showlegend=(i == 0)
        ),
        row=row, col=col
    )

    # Agregar la tendencia polinómica al subplot
    fig.add_trace(
        go.Scatter(
            x=brand_data['DATE'],
            y=brand_data['TENDENCIA'],
            mode='lines',
            name=f'Tendency Degree {i + 1}',
            line=dict(dash='dash', color='#8ecae6'),
            showlegend=(i == 0)
        ),
        row=row, col=col
    )

# Ajustar el diseño
fig.update_layout(
    title_text=f'Sales Tendency for {max_sales_brand} - Polynomial Degrees 1 to 4',
    height=800,
    width=900,
    showlegend=True
)

fig.update_xaxes(title_text="Month", tickangle=45)
fig.update_yaxes(title_text="Sales")

fig.show()

In [39]:
# Extract the sales data into a numpy array
sales = np.array(brand_data['TOTAL_SALES'])

# Perform FFT
fft_values = fft(sales)

# Calculate frequencies corresponding to the FFT values
n = len(sales)
frequencies = np.fft.fftfreq(n)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=frequencies[:n // 2],
        y=np.abs(fft_values[:n // 2]),
        mode='lines',
        line=dict(color='#457b9d'),
        name='Magnitude')
)

# Configuración del diseño del gráfico
fig.update_layout(
    title='Fourier Transform - Frequency Analysis',
    xaxis=dict(title='Frequency'),
    yaxis=dict(title='Magnitude'),
    template='plotly_white',
    width=800,
    height=500
)

# Mostrar el gráfico
fig.show()

In [40]:
# Asegúrate de que el índice sea de tipo datetime
brand_data['DATE'] = pd.to_datetime(brand_data['DATE'])

# Establecer la fecha como índice
brand_data.set_index('DATE', inplace=True)

# Realizar la descomposición estacional
result = seasonal_decompose(brand_data['TOTAL_SALES'], model='additive', period=8)

# Graficar los componentes
# result.plot()
# plt.tight_layout()
# plt.show()

# Crear la figura con subplots para cada componente
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=['Original Series', 'Trend', 'Seasonal', 'Residual'])

fig.add_trace(
    go.Scatter(x=country_data.index, y=country_data['TOTAL_SALES'], mode='lines', name='Original Series'),
    row=1, col=1)

fig.add_trace(
    go.Scatter(x=country_data.index, y=result.trend, mode='lines', name='Trend'),
    row=2, col=1)

fig.add_trace(
    go.Scatter(x=country_data.index, y=result.seasonal, mode='lines', name='Seasonal'),
    row=3, col=1)

fig.add_trace(
    go.Scatter(x=country_data.index, y=result.resid, mode='markers', name='Residual', marker=dict(color='green')),
    row=4, col=1)

# Configuración del layout del gráfico
fig.update_layout(
    title='Seasonal Decomposition of Total Sales',
    height=800,
    template='plotly_white')

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Sales", tickformat=".2f")
fig.show()

### Predicciones España y evaluación de la bondad

In [41]:
forecast_spain = df_forecast[df_forecast['COUNTRY'] == 'Spain']
actual_spain = df_actual[df_actual['COUNTRY'] == 'Spain']

In [42]:
set1 = set(actual_spain['DATE'].unique())
set2 = set(forecast_spain['DATE'].unique())

solo_en_lista1 = set1 - set2  # Elementos en lista1 que no están en lista2
solo_en_lista2 = set2 - set1  # Elementos en lista2 que no están en lista1

print("Elementos solo en lista1:", solo_en_lista1)
print("Elementos solo en lista2:", solo_en_lista2)

Elementos solo en lista1: set()
Elementos solo en lista2: {Timestamp('2024-11-01 00:00:00'), Timestamp('2025-04-01 00:00:00'), Timestamp('2025-03-01 00:00:00'), Timestamp('2025-05-01 00:00:00'), Timestamp('2024-10-01 00:00:00'), Timestamp('2025-01-01 00:00:00'), Timestamp('2024-09-01 00:00:00'), Timestamp('2025-02-01 00:00:00'), Timestamp('2024-12-01 00:00:00')}


In [43]:
# Filtrar las fechas que están en ambos DataFrames
combined_data = forecast_spain[forecast_spain['DATE'].isin(actual_spain['DATE'])].reset_index(drop=True)

# Crear una lista de gráficos por cada marca
for brand in combined_data['SUBBRAND'].unique():
    # Filtrar datos para la marca actual
    brand_data = combined_data[combined_data['SUBBRAND'] == brand]

    # Crear boxplot para predicciones
    boxplot = go.Box(
        x=brand_data['DATE'],
        y=brand_data['AMOUNT'],
        name="Predicciones",
        marker_color='#8ecae6',
        boxmean=True,
        showlegend=True
    )

    # Graficar los valores reales como puntos
    real_values = go.Scatter(
        x=actual_spain['DATE'][actual_spain['SUBBRAND'] == brand],
        y=actual_spain['AMOUNT'][actual_spain['SUBBRAND'] == brand],
        mode='markers',
        name='Valores Reales',
        marker=dict(color='#457b9d', size=8),
        showlegend=True
    )

    # Configurar diseño del gráfico
    layout = go.Layout(
        title=f'Comparación de Predicciones y Valores Reales para {brand}',
        xaxis=dict(title='Mes'),
        yaxis=dict(title='Ventas'),
        legend=dict(title="Leyenda"),
        template="plotly_white"
    )

    fig = go.Figure(data=[boxplot, real_values], layout=layout)
    fig.show()
